# Simulation: Patient Participation Tokenomics

All rights reserved, unlicensed.

Loads `config.json` and simulates tokens/tiers (0/3/6/9%). No PHI.

In [ ]:
import json, math, random
with open('config.json') as f:
    cfg = json.load(f)
tiers = cfg['tiers_percent']
thr = cfg['thresholds_tokens']
cap = cfg['cap_monthly_tokens_per_user']
tps = cfg['token_per_session']
pen = cfg['no_show_penalty_percent'] / 100.0
k   = cfg['decay_k']
months = cfg['simulation']['months']
cohorts = cfg['simulation']['cohorts']
pop = {name: int(1000 * cohorts[name]['size']) for name in cohorts}
print('Population by cohort:', pop)
